In [1]:
import os
corpus_root = r'D:\Academics\UF\CIS 4930\Loebner-logs-2003'
folders = os.listdir(corpus_root)
text = []
for folder in folders:
    files = os.listdir(corpus_root+'\\'+folder)
    for file in files:
        f = open(corpus_root+'\\'+folder+'\\'+file, "r")
        for line in f.readlines():
            text.append(line)
        f.close()

In [2]:
import nltk
import re

In [3]:
pattern = re.compile(r".*: (.*)") # for extracting the dialog
pattern2 = re.compile(r"(.*):.*") # for extracting the line info
labeled_sents = []
questions = []
words = set()
for line in text:
    #print(line)
    processed_line = re.findall(pattern,line)
    if len(processed_line) > 0:
        line_info = re.findall(pattern2, line)
        if re.search('judge', line_info[0], re.IGNORECASE) or re.search('confederate', line_info[0], re.IGNORECASE):
            label = 'HUMAN'
            #human_sents.append((processed_line[0], label))
        else:
            label = 'ROBOT'
            #machine_sents.append((processed_line[0], label))
        sents = nltk.sent_tokenize(processed_line[0])
        for sent in list(sents):
            #labeled_sents.append((sent, label))
            if '?' in sent and label == 'HUMAN':
                questions.append(sent)
            for w in nltk.word_tokenize(sent):
                if w.isalpha():
                    words.add(w.lower())
        labeled_sents.append((processed_line[0], label))

In [29]:
pronouns = ['i','you','u','they','us','we','them','he','she','him','her','me']
modals = ['can', 'could', 'may', 'might', 'will', 'would', 'shall', 'should', 'must']
from string import punctuation

In [35]:
def features(sent):
    features = {}
    tokenized_sent = [w.lower() for w in nltk.word_tokenize(sent)]
    
    for word in list(words):
        features['words({})'.format(word)] = (word in tokenized_sent)
    
    for pronoun in pronouns:
        features['pronouns'] = (pronoun in tokenized_sent)
        if features['pronouns']:
            break
    for modal in modals:
        features['modals'] = (modal in tokenized_sent)
        if features['modals']:
            break
    
    return features

In [36]:
import random
random.shuffle(labeled_sents)
featuresets = [(features(d), c) for (d,c) in labeled_sents]
#humanfeatures = [(d,c) for (d,c) in featuresets if c=='HUMAN']
#machinefeatures = [(d,c) for (d,c) in featuresets if c=='ROBOT']

In [37]:
#test_set = humanfeatures[:50] + machinefeatures[50:100]
test_set = featuresets[:100]
train_set = featuresets[100:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
#accuracy for unbiased test set: 0.64, 0.7, 0.77，0.71，0.65， 0.73
# - with T/F Words, Pronouns and Modals: 0.7, 0.73, 0.66
# - with count Words, Pronouns and Modals: 0.71, 0.70, 0.73, 0.63
#accuracy for random test set: 0.65, 0.71, 0.71, 0.74, 0.58
# - with T/F Words, Pronouns and Modals: 0.71, 0.71, 0.69, 0.71, 0.71
# - with count: 0.71, 0.65, 
# - with T/F Words, Pronouns, and Modals, and number of punctuations: 0.62, 

In [38]:
print(nltk.classify.accuracy(classifier, test_set))

0.66


In [39]:
classifier.show_most_informative_features(5)

Most Informative Features
             words(glad) = True            ROBOT : HUMAN  =     14.2 : 1.0
        words(sometimes) = True            ROBOT : HUMAN  =     13.4 : 1.0
             words(lets) = True            HUMAN : ROBOT  =     10.7 : 1.0
            words(since) = True            ROBOT : HUMAN  =     10.5 : 1.0
         punctuations(?) = 3               ROBOT : HUMAN  =     10.5 : 1.0


In [28]:
question = random.choice(questions)
print (question)
user_ans = input('Your Answer: ')
classifier.classify(features(user_ans))

But will you be going out before then?
Your Answer: I dont think so?


'HUMAN'